In [ ]:
import pydeb
import numpy

In [ ]:
# Look up the Catalogue of Life (CoL) identifier for the species of interest
taxon=pydeb.infer.Taxon.from_name('european rabbit')
#taxon=pydeb.infer.Taxon.from_name('loligo vulgaris')
m = taxon.get_model()
m.describe()

In [ ]:
# Simulate entire life span
# The age-maturity relationship from this simulation will be used as reference
delta_t= 0.01 * m.a_b
result = m.simulate(n=2*max(m.a_99, m.a_m)/delta_t, delta_t=delta_t)
t_ref, E_H_ref = result['t'], result['E_H']

print('Age at birth: %.3f d vs simulated %.3f d' % (m.a_b, numpy.interp(m.E_Hb, E_H_ref, t_ref)))
if m.E_Hj > m.E_Hb:
    print('Age at metamorphosis: %.3f d vs simulated %.3f d' % (m.a_j, numpy.interp(m.E_Hj, E_H_ref, t_ref)))
print('Age at puberty: %.3f d vs simulated %.3f d' % (m.a_p, numpy.interp(m.E_Hp, E_H_ref, t_ref)))

precision = 0.001

import plotly.offline as py
import plotly.graph_objects as go

data = [go.Scatter(x=[m.a_b*0.001, m.a_p], y=[m.a_b*0.001, m.a_p], line={'color': 'black'}, mode='lines', name='reference (x=y)')]
E_H = numpy.logspace(numpy.log10(m.E_Hb*0.001), numpy.log10(m.E_Hb), 100)
t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='before birth'))
if m.E_Hj > m.E_Hb:
    E_H = numpy.logspace(numpy.log10(m.E_Hb), numpy.log10(m.E_Hj), 100)
    t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
    data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='between birth and metamorphosis'))
E_H = numpy.logspace(numpy.log10(m.E_Hj), numpy.log10(m.E_Hp), 100)
t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='between metamorphosis (or birth) and puberty'))

layout = go.Layout(xaxis={'title': 'age from explicit simulation (d)', 'type': 'log'}, yaxis={'title': 'age from age_at_maturity (d)', 'type': 'log'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
E_m = m.E_m * m.L_i**3
data = [
    go.Scatter(x=result['t'], y=result['E'], line={'color': 'black'}, mode='lines', name='simulated reserve'),
    go.Scatter(x=[result['t'][0], result['t'][-1]], y=[E_m, E_m], line={'color': 'black', 'dash': 'dash'}, mode='lines', name='maximum')
]
layout = go.Layout(xaxis={'title': 'time (d)'}, yaxis={'title': 'reserve (J)'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

data = [
    go.Scatter(x=result['t'], y=result['E']/result['L']**3, line={'color': 'black'}, mode='lines', name='simulated reserve density'),
    go.Scatter(x=[result['t'][0], result['t'][-1]], y=[m.E_m, m.E_m], line={'color': 'black', 'dash': 'dash'}, mode='lines', name='maximum')
]
layout = go.Layout(xaxis={'title': 'time (d)'}, yaxis={'title': 'reserve density (J/cm3)'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
print('Ultimate reserve: %.3f J vs simulated %.3f J' % (m.E_m*m.L_i**3, result['E'][-1]))
print('Ultimate reserve density: %.3f J/cm3 vs simulated %.3f J/cm3' % (m.E_m, result['E'][-1]/result['L'][-1]**3))

In [ ]:
data = [
    go.Scatter(x=result['t'], y=result['L'], line={'color': 'black'}, mode='lines', name='simulated'),
    go.Scatter(x=[result['t'][0], result['t'][-1]], y=[m.L_i, m.L_i], line={'color': 'black', 'dash': 'dash'}, mode='lines', name='maximum')
]
layout = go.Layout(xaxis={'title': 'time (d)'}, yaxis={'title': 'structural length (cm)'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
print('Ultimate structural length: %.3f cm vs simulated %.3f cm' % (m.L_i, result['L'][-1]))

In [ ]:
data = [
    go.Scatter(x=result['t'], y=result['R'], line={'color': 'black'}, mode='lines', name='simulated'),
    go.Scatter(x=[result['t'][0], result['t'][-1]], y=[m.R_i, m.R_i], line={'color': 'black', 'dash': 'dash'}, mode='lines', name='maximum')
]
layout = go.Layout(xaxis={'title': 'time (d)'}, yaxis={'title': 'reproduction rate (#/d)'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
print('Ultimate reproduction rate: %.3f #/d vs simulated %.3f #/d' % (m.R_i, result['R'][-1]))
